# Symbolic Derivative experiments


In [1]:
import sys

In [2]:
import mira
from mira.metamodel import Concept, ControlledConversion, GroupedControlledConversion, Initial, NaturalConversion, Parameter, Template, TemplateModel
from mira.modeling.viz import GraphicalModel
from mira.modeling import Model
from mira.modeling.askenet.petrinet import AskeNetPetriNetModel
from torch import tensor
import torch
from pyciemss.interfaces import sample, calibrate
from pyciemss.PetriNetODE.interfaces import load_petri_model, sample_petri, calibrate_petri, setup_petri_model, load_and_sample_petri_model
from pyciemss.PetriNetODE.base import get_name
from torch import tensor
import sympy
from sympytorch import SymPyModule

# MIRA SIR Model

In [4]:
beta, gamma, S, I, R, total_population = sympy.symbols('beta, gamma, susceptible_population, I, recovered_population, total_population')

susceptible = Concept(name="susceptible_population", identifiers={"ido": "0000514"})
infected = Concept(name="I", identifiers={"ido": "0000573"}) # http://purl.obolibrary.org/obo/IDO_0000573
recovered = Concept(name="recovered_population", identifiers={"ido": "0000592"})
total_pop = 100000

S_to_I = ControlledConversion(
    controller = infected,
    subject=susceptible,
    outcome=infected,
    rate_law=beta*S*I/(S + I + R)
)
I_to_R = NaturalConversion(
    subject=infected,
    outcome=recovered,
    rate_law=gamma*I
)
tm = TemplateModel(
    templates=[S_to_I, I_to_R],
    parameters={
        'beta': Parameter(name='beta', value=0.55), # transmission rate
        'gamma': Parameter(name='gamma', value=0.2), # recovery rate
    },
    initials={
        'susceptible_population': (Initial(concept=susceptible, value=(total_pop - 1))), 
        'I': (Initial(concept=infected, value=1)),
        'recovered_population': (Initial(concept=recovered, value=0)),
    }
)

model = mira.modeling.Model(tm)

In [5]:
model.parameters

{'beta': <mira.modeling.ModelParameter at 0x14eca7220>,
 'gamma': <mira.modeling.ModelParameter at 0x14eca7310>}

## SIR DynamicalSystem

In [6]:
sir_raw = load_petri_model(model)
sir = setup_petri_model(sir_raw, 
                        0.0, 
                        start_state= {
                            k: v.value 
                            for k,v in model.template_model.initials.items()
                        }
                       )
sir

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.4950000047683716, high: 0.6050000190734863),
	gamma = Uniform(low: 0.18000000715255737, high: 0.2199999988079071),
	pseudocount = 1.0
)

In [7]:
sir.param_prior()

In [8]:
assert isinstance(getattr(sir, 'beta'), torch.Tensor)

### Convert MIRA SympyExprStr to regular Sympy Expr

In [9]:
def extract_sympy(sympy_expr_str: mira.metamodel.templates.SympyExprStr) -> sympy.Expr:
    """Convert the mira SympyExprStr to a sympy.Expr."""
    return sympy.sympify(str(sympy_expr_str), 
                         locals={str(x): x 
                                 for x in sympy_expr_str.free_symbols})


In [10]:
sir.var_order

OrderedDict([('I', <mira.modeling.Variable at 0x14eca7550>),
             ('recovered_population', <mira.modeling.Variable at 0x14eca7610>),
             ('susceptible_population',
              <mira.modeling.Variable at 0x1114cba00>)])

In [11]:
timepoints = [0.1, 0.2, 0.3, 0.4]
nsamples = 3

try:
    samples = sample_petri(sir, timepoints, nsamples)
except AttributeError as e:
    print(e)

### Confirm that symbolic fluxes can be compiled to numeric fluxes

In [12]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

        
flux = SymPyModule(expressions=[beta*S*I/(S + I + R), gamma*I])
fluxes = flux(beta=getattr(sir, 'beta'),
              gamma=getattr(sir,'gamma'),
              susceptible_population=states['susceptible_population'],
              I=states['I'],
              recovered_population=states['recovered_population'])
fluxes

tensor([0.5863, 0.1905])

### Confirm that a symbolic derivative can be compiled to numeric derivatives


In [13]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}


symbolic_derivs['susceptible_population'] = -beta*S*I/(S + I + R)
symbolic_derivs['I'] = beta*S*I/(S + I + R) - gamma*I
symbolic_derivs['recovered_population'] =  gamma*I


numeric_deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
numeric_deriv(beta=getattr(sir, 'beta'),
      gamma=getattr(sir,'gamma'),
      susceptible_population=states['susceptible_population'],
      I=states['I'],
      recovered_population=states['recovered_population'])

tensor([-0.5863,  0.3958,  0.1905])

### Confirm that Mira rate laws can be compiled to numeric derivatives

In [14]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}


symbolic_derivs['susceptible_population'] = -extract_sympy(S_to_I.rate_law)
symbolic_derivs['I'] = extract_sympy(S_to_I.rate_law) - extract_sympy(I_to_R.rate_law)
symbolic_derivs['recovered_population'] =  extract_sympy(I_to_R.rate_law)


numeric_deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
numeric_deriv(beta=getattr(sir, 'beta'),
      gamma=getattr(sir,'gamma'),
      susceptible_population=states['susceptible_population'],
      I=states['I'],
      recovered_population=states['recovered_population'])

tensor([-0.5863,  0.3958,  0.1905])

### Confirm that symbolic fluxes can be converted to numeric derivatives

In [15]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[get_name(c)] -= flux
    for p in t.produced:
        symbolic_derivs[get_name(p)] += flux

numeric_deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
numeric_deriv(beta=getattr(sir, 'beta'),
      gamma=getattr(sir,'gamma'),
      susceptible_population=states['susceptible_population'],
      I=states['I'],
      recovered_population=states['recovered_population'])

tensor([-0.5863,  0.3958,  0.1905])

### Confirm that numeric derivatives can be compiled using mira model parameter objects and initial states

In [16]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}


for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[c.data['name']] -= flux
    for p in t.produced:
        symbolic_derivs[p.data['name']] += flux

deriv = SymPyModule(expressions=symbolic_derivs.values())
deriv(**{param.key: getattr(sir, param.key) 
         for param in sir.G.parameters.values()
        },
      **states)

      

tensor([-0.5863,  0.3958,  0.1905])

### Alternative approach using mira model parameter keys instead of model parameter objects

In [17]:
states = {k: tensor(v.value) for k,v in model.template_model.initials.items()}
symbolic_derivs = {k: 0. for k in states}

for t in sir.G.transitions.values():
    flux = extract_sympy(t.template.rate_law)
    for c in t.consumed:
        symbolic_derivs[get_name(c)] -= flux
    for p in t.produced:
        symbolic_derivs[get_name(p)] += flux

deriv = SymPyModule(expressions=list(symbolic_derivs.values()))
deriv(**{k: getattr(sir, k) for k in model.parameters},
      **states)


tensor([-0.5863,  0.3958,  0.1905])

### ASKEM Model Representation

In [18]:
sir = load_and_sample_petri_model('https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json', 
                                  num_samples=3,
                                  timepoints=timepoints,
                                  compile_rate_law_p=True)
